In [1]:
import json
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from requests.exceptions import HTTPError
import time
from concurrent.futures import ThreadPoolExecutor

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth1.pkl', 'rb') as file:
    results = pickle.load(file)
for r, s in results.items():
    print(s)

Minh Tú đã đạt được thành tích gì trong chương trình Asia’ Next Top Model mùa thứ 5?
Minh Tú đã vượt qua sự sợ hãi để hoàn thành tốt phần thử thách đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng và đạt vị trí thứ 2 trong đêm chung kết của chương trình.
["Tại sao sương mù xuất hiện dày đặc ở TP HCM vào sáng 21/9?", "Sương mù xuất hiện dày đặc ở TP HCM có phải do mưa lớn mấy ngày trước không?", "Mức ô nhiễm của TP HCM có ảnh hưởng đến sự xuất hiện của sương mù không?"]
Vụ nổ tại nhà máy hóa dầu Lâm Nghi Kim Dự xảy ra tại đâu và vào lúc nào?
Vụ nổ tại nhà máy hóa dầu Lâm Nghi Kim Dự đã ảnh hưởng đến bao nhiêu người?
Vụ nổ tại nhà máy hóa dầu Lâm Nghi Kim Dự đã xảy ra vào ngày nào?
Tây Ban Nha cáo buộc ai về việc thúc giục Catalonia đòi độc lập?
Tây Ban Nha cáo buộc ai đã sử dụng mạng xã hội để thúc giục tiến hành cuộc trưng cầu dân ý đòi độc lập ở Catalonia?
Tây Ban Nha cáo buộc ai đã có vai trò trong việc thúc giục Catalonia đòi độc lập?
["Lý Hải có bao giờ hết thời trong sự ng

In [10]:
with open('../data/vietnamese_rag/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)
doc_index = {d['id']: d for d in documents}

In [11]:
import os
import re
import json
import pickle
import pandas as pd

def clean_and_normalize_json_string(json_string):
    # Remove unwanted characters and control characters
    json_string = re.sub(r'[\n\r\t]', '', json_string)  # Remove newlines, carriage returns, and tabs
    json_string = re.sub(r'\\', '', json_string)  # Remove backslashes
    json_string = re.sub(r'“|”', '"', json_string)  # Replace fancy quotes with standard quotes
    return json_string

def clean_and_normalize_data(data):
    # print(data)
    if isinstance(data, dict) and 'questions' in data:
        return data['questions']
    else:
        return data

def process_ground_truth_file(file_path, doc_index):
    with open(file_path, 'rb') as file:
        results = pickle.load(file)
    
    falsed_json_doc_id = []
    parsed_results = {}

    for doc_id, json_questions in results.items():
        try:
            parsed_questions = json.loads(json_questions)
            parsed_results[doc_id] = parsed_questions
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for document {doc_id}: {e}")
            falsed_json_doc_id.append(doc_id)
            cleaned_json_questions = clean_and_normalize_json_string(json_questions)
            try:
                parsed_questions = json.loads(cleaned_json_questions)
                parsed_results[doc_id] = parsed_questions
            except json.JSONDecodeError as e:
                print(f"Error decoding cleaned JSON for document {doc_id}: {e}")

    final_results = []

    for doc_id, questions in parsed_results.items():
        group = doc_index.get(doc_id, {}).get('group', 'Unknown')
        cleaned_questions = clean_and_normalize_data(questions)
        for q in cleaned_questions:
            final_results.append((q, group, doc_id))

    return final_results, falsed_json_doc_id

def process_all_ground_truth_files(directory, doc_index):
    all_final_results = []
    all_failed_docs = []

    for i in range(2, 3):
        file_path = os.path.join(directory, f'ground_truth{i}.pkl')
        if os.path.exists(file_path):
            final_results, failed_docs = process_ground_truth_file(file_path, doc_index)
            all_final_results.extend(final_results)
            all_failed_docs.extend(failed_docs)
        else:
            print(f"File {file_path} does not exist.")

    df = pd.DataFrame(all_final_results, columns=['question', 'Group', 'document'])
    return df, all_failed_docs
# Example usage
directory = '../data/vietnamese_rag/ground_truth_data'
df6, failed_docs = process_all_ground_truth_files(directory, doc_index)

Error decoding JSON for document ed4c139c: Extra data: line 1 column 70 (char 69)
Error decoding cleaned JSON for document ed4c139c: Extra data: line 1 column 70 (char 69)
Error decoding JSON for document 2cd628bf: Expecting value: line 1 column 1 (char 0)
Error decoding cleaned JSON for document 2cd628bf: Expecting value: line 1 column 1 (char 0)
Error decoding JSON for document dc0710cb: Expecting value: line 1 column 1 (char 0)
Error decoding cleaned JSON for document dc0710cb: Expecting value: line 1 column 1 (char 0)
Error decoding JSON for document 2760339c: Expecting value: line 1 column 1 (char 0)
Error decoding cleaned JSON for document 2760339c: Expecting value: line 1 column 1 (char 0)
Error decoding JSON for document 42b7378a: Expecting value: line 1 column 1 (char 0)
Error decoding cleaned JSON for document 42b7378a: Expecting value: line 1 column 1 (char 0)
Error decoding JSON for document ed60490c: Expecting value: line 1 column 1 (char 0)
Error decoding cleaned JSON for

In [12]:
len(failed_docs)

25